In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  

sheet_names = ["ML buyer seller", "HK buyer seller", "CO buyer seller", "URA buyer seller","OS buyer seller"]
df = pd.read_excel("./data/input/full records of each type of buyers and sellers no new lines.xlsx", sheet_name=sheet_names)

In [2]:
dict_df = dict()

dict_df['ML'] = pd.concat([
    df['ML buyer seller'][['ASSIG', 'ML_BUYER_LIST1']].rename(columns = {"ASSIG":"name",'ML_BUYER_LIST1':"ML"}),
    df['ML buyer seller'][['ASSGO', 'ML_SELLER_LIST1']].rename(columns = {"ASSGO":"name",'ML_SELLER_LIST1':"ML"})
])

dict_df['HK'] = pd.concat([
    df['HK buyer seller'][['ASSIG', 'HK_BUYER_LIST1']].rename(columns = {"ASSIG":"name",'HK_BUYER_LIST1':"HK"}),
    df['HK buyer seller'][['ASSGO', 'HK_SELLER_LIST1']].rename(columns = {"ASSGO":"name",'HK_SELLER_LIST1':"HK"})
])

dict_df['CO'] = pd.concat([
    df['CO buyer seller'][['ASSIG', 'COMPANY_BUYER']].rename(columns = {"ASSIG":"name",'COMPANY_BUYER':"CO"}),
    df['CO buyer seller'][['ASSGO', 'COMPANY_SELLER']].rename(columns = {"ASSGO":"name",'COMPANY_SELLER':"CO"})
])

def extract_df(df, type):
    out = df[df[type]==1] 
    out = out[['name']]
    out['class'] = type
    return out

raw_names = pd.concat([
    extract_df(dict_df[x], x) for x in ['HK','ML','CO']
])



## Data cleaning

In [3]:
step1 = raw_names.drop_duplicates()
step1.loc[:,'orig_name'] = step1['name'].str.upper()
step1 = step1[['orig_name','name','class']]

## Remove (TC) or (JT) in names

step1.name.str.extract('.*\((.*)\).*')[0].value_counts().to_csv("preview/step1_in.csv")

step1.name = step1.name.str.replace(r'(TC)','',regex=False)
step1.name = step1.name.str.replace(r'(JT)','',regex=False)
step1[step1.name.str.contains(r'\(')]
step1.name.str.extract('.*\((.*)\).*')[0].value_counts()

step1.name.str.extract('.*\((.*)\).*')[0].value_counts().to_csv("preview/step1_out.csv")


C:\Users\kin24\AppData\Local\conda\conda\envs\name_class\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\kin24\AppData\Local\conda\conda\envs\name_class\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
## Remove the shares of the house 

# with numbers
step1.name[ step1.name.str.contains(r'[0-9]{1,}')  ].to_csv("preview/step2_in1.csv")

## replace format xx/xx with ""
step1.name[ step1.name.str.contains(r'[0-9]{1,}/[0-9]{1,}') ].to_csv("preview/step2_in2.csv")

## number without /
step1.name[ step1.name.str.contains(r'[0-9]{1,}') &  ~step1.name.str.contains(r'/{1,}') ].to_csv("preview/step2_in3.csv")

## replace % with ""
step1.name[ step1.name.str.contains(r'%') ].to_csv("preview/step2_in4.csv")

## replace - with " "
step1.name[ step1.name.str.contains(r'-') ].to_csv("preview/step2_in5.csv")

In [5]:
step2 = step1

step2.name = step2.name.str.replace(r'[0-9]{1,}/[0-9]{1,}','',regex=True)
step2.name = step2.name.str.replace(r'[0-9]{1,}/','',regex=True)
step2.name = step2.name.str.replace(r'/[0-9]{1,}','',regex=True)

step2.name = step2.name.str.replace(r'[0-9]{1,}%','',regex=True)
step2.name = step2.name.str.replace(r'[0-9]{1,}\.[0-9]{1,}%','',regex=True)

step2.name = step2.name.str.replace(r'-',' ',regex=False)


In [6]:
## Remove the shares of the house 

# with numbers
step2.name[ step2.name.str.contains(r'[0-9]{1,}')  ].to_csv("preview/step2_out1.csv")

## replace format xx/xx with ""
step2.name[ step2.name.str.contains(r'[0-9]{1,}/[0-9]{1,}') ].to_csv("preview/step2_out2.csv")

## number without /
step2.name[ step2.name.str.contains(r'[0-9]{1,}') &  ~step2.name.str.contains(r'/{1,}') ].to_csv("preview/step2_out3.csv")

## replace % with ""
step2.name[ step2.name.str.contains(r'%') ].to_csv("preview/step2_out4.csv")

## replace - with " "
step2.name[ step2.name.str.contains(r'-') ].to_csv("preview/step2_out5.csv")

In [7]:
## Replace the remaining numbers with " "
step3 = step2 

step3.name[ step3.name.str.contains(r'[0-9]{1,}')  ].to_csv("preview/step3_in1.csv")

step3.name = step3.name.str.replace(r'[0-9]{1,}',' ',regex=True)
step3.name = step3.name.str.replace(r' {1,}',' ',regex=True)


step3.name[ step3.name.str.contains(r'[0-9]{1,}')  ].to_csv("preview/step3_out1.csv")


In [8]:
step3.name[ step3.name.str.contains(r'[^A-Z\s]{1,}')  ].to_csv("preview/step4_in1.csv")


## Check for duplication

In [9]:
tmp = step3
tmp['count'] = tmp.groupby('orig_name')['orig_name'].transform('count')
tmp[tmp['count']>1].to_csv('preview/duplicates.csv')

In [10]:
## Most of the duplication are caused by company. We will replace all the values with CO

In [11]:
step4 = step3
step4['count'] = step4.groupby('orig_name')['orig_name'].transform('count')

step4.loc[step4['count']>1,'class'] = 'ML'
step4 = step4.drop_duplicates()

In [12]:
step4.to_parquet('./data/output/raw_names.parquet', compression = None)